In [1]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install scikit-learn
%pip install openpyxl



You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


  Using cached matplotlib-3.5.3-cp37-cp37m-win_amd64.whl (7.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: matplotlib 3.5.3 requires cycler>=0.10, which is not installed.
ERROR: matplotlib 3.5.3 requires pillow>=6.2.0, which is not installed.
ERROR: matplotlib 3.5.3 requires pyparsing>=2.2.1, which is not installed.
You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


  Using cached scikit_learn-1.0.2-cp37-cp37m-win_amd64.whl (7.1 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [2]:
# Code to allow GPU access
import tensorflow.compat.v1 as tf 

In [3]:
'''LOADING DATA FROM FILES'''

import pandas as pd
import os

# Load label file
label_df = pd.read_excel('../Data/wax/data labels anna - simple.xlsx')
# clean up column names
label_df.columns = ['code', 'month', 'ID', 'optimality', 'abnormal', 'AIMS']
print(label_df.head())

# Cleanup data, remove rows where AIMS score is 999
#cleaned_label_df = label_df[label_df.AIMS != 999]


   code  month   ID  optimality  abnormal  AIMS
0  1023      0  180           9       1.0     1
1  1023      1  195          14       1.0   999
2  1023      2  197          26       1.0     1
3  1023      3  206          11       1.0     2
4  1023      4  208          13       1.0     1


In [4]:
'''LOADING SENSOR DATA INTO DATAFRAMES and TRIMMING TRIALS'''

import matplotlib.pyplot as plt
import numpy as np

def load_sensor_data(trial_folder, sensor):
    df = pd.read_csv(trial_folder + "/" + sensor + ".csv")
    df = df[(df.T != 0).any()]
    return df

'''def truncate_df(df, min_rows):
    return df[:min_rows]'''

def combine_dfs(dfs):
    return pd.concat(dfs, axis=1)

def process_df(df, min_rows):
    # Drop rows where all entries are zero
    df = df[(df.T != 0).any()]
    # Truncate dataframe to min_rows
    df = df[:min_rows]
    return df

def rename_columns(df):
    df.columns = ['Time', 'AccXRA', 'AccYRA', 'AccZRA', 'GyrXRA', 'GyrYRA', 'GyrZRA', 'MagXRA', 'MagYRA', 'MagZRA',
                  'AccXLA', 'AccYLA', 'AccZLA', 'GyrXLA', 'GyrYLA', 'GyrZLA', 'MagXLA', 'MagYLA', 'MagZLA',
                  'AccXRW', 'AccYRW', 'AccZRW', 'GyrXRW', 'GyrYRW', 'GyrZRW', 'MagXRW', 'MagYRW', 'MagZRW',
                  'AccXLW', 'AccYLW', 'AccZLW', 'GyrXLW', 'GyrYLW', 'GyrZLW', 'MagXLW', 'MagYLW', 'MagZLW']
    return df

def calculate_acceleration_sums(df):
    df['AccSumRA'] = np.sqrt(df['AccXRA']**2 + df['AccYRA']**2 + df['AccZRA']**2)
    df['AccSumLA'] = np.sqrt(df['AccXLA']**2 + df['AccYLA']**2 + df['AccZLA']**2)
    df['AccSumRW'] = np.sqrt(df['AccXRW']**2 + df['AccYRW']**2 + df['AccZRW']**2)
    df['AccSumLW'] = np.sqrt(df['AccXLW']**2 + df['AccYLW']**2 + df['AccZLW']**2)
    df['AccSumSumR'] = df['AccSumRA'] + df['AccSumRW']+ df['AccSumLA'] + df['AccSumLW']
    return df

def calculate_rolling_mean(df, window_size):
    df['AccSumSumR_Rolling10000'] = df['AccSumSumR'].rolling(window_size).mean()
    return df

def trim_df(df, start_index, end_index):
    return df.iloc[start_index:end_index-10000]

def drop_columns(df, columns):
    df.drop(columns=columns, inplace=True)
    return df

def calculate_peak_indices(first_half, second_half, column_name):
    start_peak_index = first_half[column_name].idxmax() + 10000
    end_peak_index = second_half[column_name].idxmax() - 10000
    return start_peak_index, end_peak_index

def process_trial(index, row, label_df):

    print("processing instance", index)
    trial_id = row["ID"]
    infant_id = row["code"]
    if trial_id < 10:
        addedZero = '00'
    elif row['ID'] < 100:
        addedZero = '0'
    else:
        addedZero = ''

    trial_folder = '../Data/wax/p' + addedZero + str(trial_id)

    ra_df = load_sensor_data(trial_folder, "RA")
    la_df = load_sensor_data(trial_folder, "LA")
    rw_df = load_sensor_data(trial_folder, "RW")
    lw_df = load_sensor_data(trial_folder, "LW")

    # Get the minimum number of rows across all dataframes
    min_rows = min(ra_df.shape[0], la_df.shape[0], rw_df.shape[0], lw_df.shape[0])  

    #Process each dataframe
    ra_df = process_df(ra_df, min_rows)
    la_df = process_df(la_df, min_rows)
    rw_df = process_df(rw_df, min_rows)
    lw_df = process_df(lw_df, min_rows)

    # Remove first column from la, rw, lw dfs using indexing
    la_df = la_df.iloc[:, 1:]
    rw_df = rw_df.iloc[:, 1:]
    lw_df = lw_df.iloc[:, 1:]

    combined_df = combine_dfs([ra_df, la_df, rw_df, lw_df])
    combined_df = rename_columns(combined_df)
    combined_df = calculate_acceleration_sums(combined_df)

    # Split df in half
    first_half = combined_df.iloc[0:combined_df.shape[0]//2]
    second_half = combined_df.iloc[combined_df.shape[0]//2:combined_df.shape[0]]
   
    ra_start_peak_index, ra_end_peak_index = calculate_peak_indices(first_half, second_half, 'AccSumRA')
    la_start_peak_index, la_end_peak_index = calculate_peak_indices(first_half, second_half, 'AccSumLA')
    rw_start_peak_index, rw_end_peak_index = calculate_peak_indices(first_half, second_half, 'AccSumRW')
    lw_start_peak_index, lw_end_peak_index = calculate_peak_indices(first_half, second_half, 'AccSumLW')

    starts = [ra_start_peak_index, la_start_peak_index, rw_start_peak_index, lw_start_peak_index]
    ends = [ra_end_peak_index, la_end_peak_index, rw_end_peak_index, lw_end_peak_index]

    max_end = max(ends)
    max_index = ends.index(max_end)
    max_start = starts[max_index]
    difference = max_end - max_start

    ra_df_trimmed = trim_df(ra_df, ra_start_peak_index, ra_start_peak_index + difference)
    la_df_trimmed = trim_df(la_df, la_start_peak_index, la_start_peak_index + difference)
    rw_df_trimmed = trim_df(rw_df, rw_start_peak_index, rw_start_peak_index + difference)
    lw_df_trimmed = trim_df(lw_df, lw_start_peak_index, lw_start_peak_index + difference) 
    
    # Reset the index of each df
    ra_df_trimmed = ra_df_trimmed.reset_index(drop=True)
    la_df_trimmed = la_df_trimmed.reset_index(drop=True)
    rw_df_trimmed = rw_df_trimmed.reset_index(drop=True)
    lw_df_trimmed = lw_df_trimmed.reset_index(drop=True)

    trimmed_df = combine_dfs([ra_df_trimmed, la_df_trimmed, rw_df_trimmed, lw_df_trimmed])
    trimmed_df = rename_columns(trimmed_df)
    trimmed_df = calculate_acceleration_sums(trimmed_df)

    # Plot trimmed_df for every 10 instances (sanity check)
    if index % 10 == 0:
        trimmed_df.plot(x='Time', y=['AccSumRA', 'AccSumLA', 'AccSumRW', 'AccSumLW'], title='Sum of Accelerometer Values for each limb (Data trimmed)')
    trimmed_df = drop_columns(trimmed_df, ['AccSumRA', 'AccSumLA', 'AccSumRW', 'AccSumLW', 'AccSumSumR'])

    # If trimmed_df is empty, raise an exception. Prevents empty dataframes from being added to the list
    if trimmed_df.empty:
        raise Exception("Empty dataframe")

    return trimmed_df, row['month'], row['code'], row['abnormal'], row['AIMS'], row['optimality']

sensorX = [] # List of dataframes (just the sensor data)
accelX = [] # List of dataframes (just the accelerometer data)
combinedX = [] # Sensor data, infant ID, infant age
abnormalY = [] # Abnormal / Normal
aimsY = [] # AIMS score
optimalityY = [] # Optimality score

for index, row in label_df.iterrows():
    try:
        combined_df_trimmed, month, code, abnormal, aims, optimality = process_trial(index, row, label_df)

        # Add all sensor data to sensorX
        sensorX.append(combined_df_trimmed)

        # Add all sensor data, infant ID, infant age to combinedX
        combinedX.append([combined_df_trimmed, month, code])

        # Add just accelerometer data to accelX
        combined_df_trimmed_copy = combined_df_trimmed.copy()
        combined_df_trimmed_copy = drop_columns(combined_df_trimmed_copy, ['MagXRA', 'MagYRA', 'MagZRA', 
                                                                           'MagXLA', 'MagYLA', 'MagZLA', 
                                                                           'MagXRW', 'MagYRW', 'MagZRW', 
                                                                           'MagXLW', 'MagYLW', 'MagZLW']) # drop magnetometer data
        combined_df_trimmed_copy = drop_columns(combined_df_trimmed_copy, ['GyrXRA', 'GyrYRA', 'GyrZRA', 
                                                                           'GyrXLA', 'GyrYLA', 'GyrZLA', 
                                                                           'GyrXRW', 'GyrYRW', 'GyrZRW', 
                                                                           'GyrXLW', 'GyrYLW', 'GyrZLW']) # drop gyroscope data    
        accelX.append(combined_df_trimmed_copy)
        
        # Add abnormality label to abnormalY
        abnormalY.append(abnormal)

        # Add AIMS score to aimsY
        aimsY.append(aims)

        # Add optimality score to optimalityY
        optimalityY.append(optimality)

    except FileNotFoundError:
        print("File not found for instance", index)

    except Exception as e:
        print("Error processing instance", index)
        print("Error", e)   

# Sanity check: Print results
print("Sensor X:")
print(sensorX[0])
print("Combined X:")
print(combinedX[0])
print("Accel X:")
print(accelX[0])
print("Labels")
print(abnormalY)
print(aimsY)
print(optimalityY)

processing instance 0
File not found for instance 0
processing instance 1
processing instance 2
File not found for instance 2
processing instance 3
File not found for instance 3
processing instance 4
File not found for instance 4
processing instance 5
File not found for instance 5
processing instance 6
File not found for instance 6
processing instance 7
processing instance 8
processing instance 9
processing instance 10
processing instance 11
File not found for instance 11
processing instance 12
processing instance 13
processing instance 14
Error processing instance 14
Error Empty dataframe
processing instance 15
processing instance 16
processing instance 17
File not found for instance 17
processing instance 18
File not found for instance 18
processing instance 19
processing instance 20
processing instance 21
processing instance 22
File not found for instance 22
processing instance 23
File not found for instance 23
processing instance 24
processing instance 25
processing instance 26
pro

In [2]:
'''SCALING ACCELEROMETER DATA'''

# Make each dataframe the same length in accelX
# Find the minimum length
min_length = min([df.shape[0] for df in accelX])
print("Minimum length:", min_length)

# Truncate each dataframe to the minimum length
accelX_clean = [df[:min_length] for df in accelX]

# Convert accelX to a numpy array
accelX_clean = np.array([df.values for df in accelX_clean])

# Convert accelX to a tensor
#%pip install torch
import torch
accelX_clean = torch.Tensor(accelX_clean)

print("Accel X shape:", accelX_clean.shape)

# Normalise accelX
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
accelX_scaled = scaler.fit_transform(accelX_clean.reshape(-1, accelX_clean.shape[-1])).reshape(accelX_clean.shape)

print("Accel X shape:", accelX_scaled.shape)
print(accelX_scaled[0])


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\User\OneDrive - University of Glasgow\Documents\5th Year\Abnormal Infant Movement Detection\Abnormal-Infant-Movement-Detection\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  File "C:\Users\User\AppData\Local\Temp\ipykernel_3028\2787775637.py", line 5, in <module>
    min_length = min([df.shape[0] for df in accelX])
                                            ^^^^^^
NameError: name 'accelX' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\User\OneDrive - University of Glasgow\Documents\5th Year\Abnormal Infant Movement Detection\Abnormal-Infant-Movement-Detection\.venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  

In [6]:
'''CREATE TEST AND TRAINING SETS'''

# X dataset : accelX_scaled
# Y dataset : abnormalY

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(accelX_scaled, abnormalY, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", len(Y_train))
print("Y_test shape:", len(Y_test))

# Convert Y_train and Y_test to tensors
Y_train = torch.Tensor(Y_train)
Y_test = torch.Tensor(Y_test)

X_train shape: (117, 10885, 13)
X_test shape: (30, 10885, 13)
Y_train shape: 117
Y_test shape: 30


In [1]:
# Save test and train sets to files
torch.save(X_train, '../Data/wax/trainAndTest/X_train.pt')
torch.save(X_test, '../Data/wax/trainAndTest/X_test.pt')
torch.save(Y_train, '../Data/wax/trainAndTest/Y_train.pt')
torch.save(Y_test, '../Data/wax/trainAndTest/Y_test.pt')


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\User\OneDrive - University of Glasgow\Documents\5th Year\Abnormal Infant Movement Detection\Abnormal-Infant-Movement-Detection\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  File "C:\Users\User\AppData\Local\Temp\ipykernel_3028\2696977367.py", line 2, in <module>
    torch.save(X_train, '../Data/wax/trainAndTest/X_train.pt')
    ^^^^^
NameError: name 'torch' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\User\OneDrive - University of Glasgow\Documents\5th Year\Abnormal Infant Movement Detection\Abnormal-Infant-Movement-Detection\.venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\User\OneDrive - U

In [9]:
# Create simple keras CNN model to test data
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

AttributeError: module 'inspect' has no attribute 'ArgSpec'

In [8]:
# Enable GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))


GPU device not found


In [1]:
# Use Keras to build a model. Start with CNN.
#%pip install keras
#%pip install tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers

# Build the model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(accelX_scaled.shape[1], accelX_scaled.shape[2])))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
print(model.summary())

AttributeError: module 'inspect' has no attribute 'ArgSpec'

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)


In [3]:
# Normalise the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#scaler.fit(accelX)

print(accelX)
print(sensorX)


# Visualise the normalised data
normalised_accelX = []
for df in accelX:
    print(type(df))
    scaler.fit(df)
    normalised_df = scaler.transform(df)
    print(type(normalised_df))
    normalised_accelX.append(normalised_df)
    #change to dataframe
    print(normalised_df)


[         Time   AccXRA   AccYRA   AccZRA   AccXLA   AccYLA   AccZLA   AccXRW  \
0      231.57  0.07910  0.02148 -0.98242  0.07031  0.49902 -0.85840 -0.89062   
1      231.58  0.08691  0.02344 -0.98828  0.07031  0.50391 -0.86035 -0.90430   
2      231.59  0.08691  0.02344 -0.97949  0.07129  0.50488 -0.86133 -0.91797   
3      231.60  0.08398  0.02637 -0.99023  0.07129  0.50488 -0.86133 -0.91797   
4      231.61  0.07324  0.02637 -0.96387  0.06738  0.50488 -0.86133 -0.89355   
...       ...      ...      ...      ...      ...      ...      ...      ...   
59033  821.90 -1.06250  0.13379  0.21875  1.54492 -0.17383  0.76270  0.67969   
59034  821.91 -1.06250  0.12598  0.20996  1.34668 -0.26562  0.76270  0.66504   
59035  821.92 -1.05566  0.11328  0.19629  0.81348 -0.26562  0.85840  0.66406   
59036  821.93 -1.04590  0.10352  0.17480  0.81348 -0.26562  0.87207  0.65723   
59037  821.94 -1.02539  0.10352  0.14941  0.75977 -0.17383  0.87207  0.65723   

        AccYRW   AccZRW   AccXLW   Acc

ValueError: Found array with 0 sample(s) (shape=(0, 13)) while a minimum of 1 is required by MinMaxScaler.

In [7]:
from sklearn.preprocessing import Normalizer

# AccelX is currently a list of pandas dataframes. Covert it to a form that can be used by the normaliser
accelX1 = [df.values.flatten() for df in accelX]


# Fit the normaliser to accelX and transform
normaliser = Normalizer()
normaliser.fit(accelX1)
normalised_accelX = normaliser.transform(accelX1)
print(normalised_accelX)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [10]:
from sklearn.preprocessing import Normalizer

# Find the maximum length of the flattened arrays
max_len = max(df.size for df in accelX)

# Flatten each dataframe and pad shorter arrays with zeros
accelX1 = [np.pad(df.values.flatten(), (0, max_len - df.size)) for df in accelX]

# Convert accelX1 to a numpy array
accelX1 = np.array(accelX1)

# 

# Fit the normaliser to accelX1 and transform
normaliser = Normalizer()
normaliser.fit(accelX1)
normalised_accelX = normaliser.transform(accelX1)
print(normalised_accelX)

# Assuming that 'AccXRA', 'AccYRA', 'AccZRA' are the first three columns
plt.plot(normalised_accelX[0][:, 0], label='AccXRA')
plt.plot(normalised_accelX[0][:, 1], label='AccYRA')
plt.plot(normalised_accelX[0][:, 2], label='AccZRA')

plt.title('Right Ankle Accelerometer Data (Normalised)')
plt.legend()
plt.show()

[[ 1.72141902e-03  5.88004682e-07  1.59675608e-07 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.17592884e-03  1.45043551e-06  2.25945799e-06 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.19981895e-04 -1.64953350e-06  1.04618377e-06 ...  1.56250334e-06
  -9.68836823e-07 -6.05270538e-07]
 [ 5.01414437e-04  2.32366173e-07  5.61533409e-08 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [34]:
# Use PCA for dimensionality reduction for combined X
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(sensorX)
X_pca = pca.transform(sensorX)
print("Original shape:   ", sensorX.shape)
print("Transformed shape:", X_pca.shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [ ]:
# Use PCA for dimensionality reduction for sensor X
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(combinedX)

NotFittedError: This PCA instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.